We have previously shown how to use OpenMM to run molecular dynamics (MD) simulations
with [molecular mechanics (MM)](https://en.wikipedia.org/wiki/Molecular_mechanics) force fields.

Despite remarkable speed ⚡️, MM oversimplifies the interactions among molecules as
low-order Fourier series, and harmonic and pairwise terms, and might not be accurate enough
on high energy region.

![MM](mm.png)

In this notebook, we replace the traditional MM in `OpenMM` with the popular
[machine learning potential](https://en.wikipedia.org/wiki/Machine_learning_potential) 
that have been rapidly developed in recent years.

First, we need to intall the `openmmml` package using `conda install -c conda-forge openmm-ml` to complement the `OpenMM` package with
ML potential functionality.

### First simulation with OpenMM-ML

In [2]:
import openmm
from openmm import app, unit
from openmmml import MLPotential

The [PDB](https://www.rcsb.org) loading stage is identical to the original `OpenMM` procedure.
Here we again load the good old alanine dipeptide example. 

In [3]:
import urllib
urllib.request.urlretrieve(
    "https://raw.githubusercontent.com/openmm/openmm/master/wrappers/python/tests/systems/alanine-dipeptide-explicit.pdb",
    "input.pdb"
)
pdb = app.PDBFile('input.pdb')

In lieu of a (MM) foce field, we here specify a machine learning potential.

In [4]:
potential = MLPotential('ani2x')

The creation of the system is again very similar to MM.
Note that here we don't need to specify specific solvent model etc. since the entire system
is treated without distinction by the ML potential.
In other words, the machine learning potential only sees a point cloud, without
paying attention to which part is solvent, protein, or ligand.

In [5]:
system = potential.createSystem(pdb.topology)

/Users/yuanqingwang/miniconda3/envs/openmm/lib/python3.11/site-packages/torchani/aev.py:16: UserWarning: cuaev not installed
  warnings.warn("cuaev not installed")
/Users/yuanqingwang/miniconda3/envs/openmm/lib/python3.11/site-packages/torchani/__init__.py:55: UserWarning: Dependency not satisfied, torchani.ase will not be available
  warnings.warn("Dependency not satisfied, torchani.ase will not be available")


/Users/yuanqingwang/miniconda3/envs/openmm/lib/python3.11/site-packages/torchani/resources/
failed to equip `nnpops` with error: No module named 'NNPOps'


Now, to construct an integrator and a simulation is just like before.

In [6]:
integrator = openmm.LangevinIntegrator(
    300*unit.kelvin, 
    1.0/unit.picoseconds, 
    2.0*unit.femtoseconds,
)

In [7]:
simulation = app.Simulation(pdb.topology, system, integrator)

In [8]:
simulation.context.setPositions(pdb.positions)

In [9]:
simulation.minimizeEnergy(maxIterations=10)